In [40]:
import os
import json

with open('source_logical_error_rate.json', 'r') as f:
    source_data = json.load(f)

with open('output_logical_error_rate.json', 'r') as f1:
    output_data = json.load(f1)

In [60]:
import numpy as np

PerfectSourceList = []
NoneChangeList = []
EnhanceList = []
WorseList = []
CompileErrorList = []

for source_key, source_value in source_data.items():
    source_problem = int(source_key.split("_")[1])
    source_index = int(source_key.split("_")[3])
    # print(f'{source_problem} {source_index}')
    source_error_rate = source_value[0]
    if source_error_rate == 0.0:
        PerfectSourceList.append([source_problem, source_index])
        continue
    
    for index, output in enumerate(output_data.items()):
        output_key, output_value = output
        output_problem = int(output_key.split("_")[1])
        output_index = int(output_key.split("_")[3])
        # print(f'{output_problem} {output_index}')
        output_error_rate = output_value[0]

        if (source_problem == output_problem) & (source_index == output_index):
            if(output_error_rate == source_error_rate):
                NoneChangeList.append([source_problem, output_index])
            elif (output_error_rate < source_error_rate):
                EnhanceList.append([source_problem, output_index, output_error_rate, source_error_rate])
            else:
                WorseList.append([source_problem, output_index, output_error_rate, source_error_rate])
            break

        if index == len(output_data) - 1:
            CompileErrorList.append([source_problem, output_index])

In [62]:
print(len(PerfectSourceList))
print(len(NoneChangeList))
print(len(EnhanceList))
print(len(WorseList))
print(len(CompileErrorList))

202
324
125
152
45


In [53]:
print(len(source_data))
print(len(output_data))

848
794


In [63]:
print(f"Fix ratio {len(EnhanceList)/(len(source_data) - len(PerfectSourceList))}")

Fix ratio 0.19349845201238391


In [58]:
print(EnhanceList)

[[0, 1, 0.0, 0.010000000000000009], [102, 530, 0.0, 0.44999999999999996], [105, 537, 0.0, 0.995], [106, 543, 0.0, 1.0], [106, 548, 0.0, 1.0], [108, 562, 0.010000000000000009, 0.35], [10, 29, 0.0, 0.0050000000000000044], [115, 591, 0.0, 0.0050000000000000044], [117, 595, 0.375, 0.39], [117, 598, 0.37, 0.39], [119, 600, 0.945, 1.0], [119, 601, 0.62, 0.745], [119, 603, 0.65, 0.665], [131, 627, 0.625, 0.72], [133, 635, 0.0, 1.0], [136, 645, 0.0, 0.0050000000000000044], [139, 663, 0.975, 1.0], [139, 668, 0.0, 0.96], [139, 669, 0.0, 0.96], [139, 671, 0.0, 0.96], [139, 672, 0.0, 0.96], [139, 673, 0.0, 0.985], [13, 40, 0.0, 0.28], [13, 41, 0.0, 1.0], [13, 42, 0.0, 1.0], [13, 44, 0.0, 1.0], [13, 46, 0.0, 1.0], [13, 52, 0.0, 0.28], [13, 55, 0.0, 0.28], [141, 678, 0.0, 0.015000000000000013], [141, 680, 0.0, 0.595], [148, 761, 0.07499999999999996, 0.09499999999999997], [155, 798, 0.0, 0.8064516129032258], [157, 811, 0.0, 0.995], [158, 713, 0.45499999999999996, 1.0], [158, 816, 0.45499999999999996,

In [64]:
Perfect_Fix = []
Semi_Fix = []
for data in EnhanceList:
    if data[0] == 0.0:
        Perfect_Fix.append(data[0:1])
    else:
        Semi_Fix.append(data[0:1])

print(f'Perfect Fix Ratio {len(Perfect_Fix)/(len(source_data) - len(PerfectSourceList))}')
print(f'Semi Fix Ratio {len(Semi_Fix)/(len(source_data) - len(PerfectSourceList))}')

Perfect Fix Ratio 0.0015479876160990713
Semi Fix Ratio 0.19195046439628483
